## Download and process TerraClimate's Climatic Water Deficit

Also pre-process GLEAM's root-zone soil moisture

In [ ]:
import os
import ee
import geemap
import warnings
import xarray as xr
import pandas as pd
import numpy as np

from odc.geo.xr import assign_crs

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [ ]:
product = 'IDAHO_EPSCOR/TERRACLIMATE'
bands = ['def']
start_date = '1/1/1982'
end_date = '12/31/2023'
output_crs = "EPSG:3577"
resolution = 5000

# ~Aus region
region = ee.Geometry.Polygon([[
            [110,-43.0],
            [155,-43.0],
            [155,-9.0],
            [110,-9.0],
            [110,-43.0]]])


In [ ]:
#use start and end dates to loop through months and load GEE data
start = pd.date_range(start=start_date, end=end_date, freq='MS') 
end = pd.date_range(start=start_date, end=end_date, freq='ME')

i = 0
for s, e in zip(start,end):
    print(" {:03}/{:03}\r".format(i+1, len(start)), end="")
    ss = s+pd.Timedelta(14, 'd')
    s = s.strftime('%Y-%m-%d')
    e = e.strftime('%Y-%m-%d')

    ds = (ee.ImageCollection(product) 
      .filterDate(s,e) 
      .map(lambda image: image.clip(region))
      .select(bands)
      .map(lambda image: image.multiply(0.1))
      .reduce(ee.Reducer.median())
         )
    
    export = ee.batch.Export.image.toDrive(ds,
        description='TerraClimate_CWD'+'_'+ss.strftime('%Y-%m-%d'),
        folder='TerraClimate_CWD',
        region=region,
        scale=resolution,
        crs=output_crs,
        maxPixels=1e13,
        fileFormat='GeoTIFF',
        formatOptions={
            'cloudOptimized':True
              }
        )
    export.start()
    i+=1


## Plot a map

In [ ]:
# Using basemap in geemap
Map = geemap.Map(center=[-35.2041, 149.2721], zoom=4)

ndvi_palette=['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']

# ndvi_palette = [
#     'a50026', 'd73027', 'f46d43', 'fdae61', 'fee08b', 'ffffbf',
#     'd9ef8b', 'a6d96a', '66bd63', '1a9850', '006837',
#   ]

ndviVis = {'min': 10, 'max': 500, 'palette': ndvi_palette}

Map.addLayer(ds, ndviVis)
Map

## Stitch together geotiffs

In [ ]:
import os
import pickle
import xarray as xr
import numpy as np
import pandas as pd
from odc.geo.xr import assign_crs
import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import round_coords

In [ ]:
base='/g/data/os22/chad_tmp/Aus_CO2_fertilisation/data/TerraClimate_CWD/'
name= 'def'
dates_start = '1982-01-01'
dates_end = '2023-12-31'

In [ ]:
files = [base+i for i in os.listdir(base) if i.endswith('.tif')]
files.sort()

time = (pd.date_range(dates_start, dates_end, freq='MS') + pd.DateOffset(days=14)).to_numpy()
time = list(time)

print(len(files), print(len(time)))

In [ ]:
i=0
arrs=[]
for f, t in zip(files, time):
    print(" {:02}/{:02}\r".format(i + 1, len(files)), end="")
    
    ds = xr.open_dataarray(f).squeeze().drop_vars('band').astype('float32')
    ds = ds.expand_dims(time=[t])
    ds = ds.rename(name)
    
    del ds.attrs['long_name']
    del ds.attrs['AREA_OR_POINT']

    ds.attrs['nodata']=np.nan
    ds = ds.drop_vars('spatial_ref')
    ds = assign_crs(ds, crs='epsg:3577')
    arrs.append(ds)

    i+=1
    

In [ ]:
ds = xr.concat(arrs, dim='time').sortby('time')

In [ ]:
with open(f'/g/data/xc0/project/AusEFlux/data/grid_5km', 'rb') as f:
    gbox_5km = pickle.load(f)

mask = xr.open_dataset(f'/g/data/xc0/project/AusEFlux/data/land_sea_mask_5km.nc')['landsea_mask_5km']

In [ ]:
ds = ds.odc.reproject(how=gbox_5km, resampling='bilinear')

In [ ]:
ds = round_coords(ds)
ds = ds.where(mask)
ds.name='CWD'

In [ ]:
ds.astype('float32').to_netcdf('/g/data/os22/chad_tmp/Aus_CO2_fertilisation/data/CWD_TerraClimate_5km_1982_2023.nc')

## Add CWD to covars file

In [ ]:
import xarray as xr

In [ ]:
covar_file = f'/g/data/os22/chad_tmp/Aus_phenology/data/covars.nc'

In [ ]:
covariables =  xr.open_dataset(covar_file)
cwd = xr.open_dataset('/g/data/os22/chad_tmp/Aus_CO2_fertilisation/data/CWD_TerraClimate_5km_1982_2023.nc')
cwd = cwd.sel(time=slice('1982','2022'))
covariables['cwd'] = cwd['CWD']

In [ ]:
covariables.to_netcdf('/g/data/os22/chad_tmp/Aus_CO2_fertilisation/data/covars.nc')

## GLEAM root-zone SM

In [ ]:
import os
import pickle
import xarray as xr
import numpy as np
import pandas as pd
from odc.geo.xr import assign_crs
import sys
sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _utils import round_coords

In [ ]:
base='/g/data/os22/chad_tmp/Aus_CO2_fertilisation/data/SMrz/'
name= 'SMrz'
dates_start = '1980-01-01'
dates_end = '2023-12-31'

In [ ]:
with open(f'/g/data/xc0/project/AusEFlux/data/grid_5km', 'rb') as f:
    gbox = pickle.load(f)

mask = xr.open_dataset(f'/g/data/xc0/project/AusEFlux/data/land_sea_mask_5km.nc')['landsea_mask_5km']

files = [f'{base}{i}' for i in os.listdir(base) if i.endswith(".nc")]
files.sort()

In [ ]:
# #combine annual files into one file
ds = xr.open_mfdataset(files)
ds = ds.sel(lat=slice(-10,-45), lon=slice(111,155))
ds = ds.rename({'lat':'latitude', 'lon':'longitude'})
ds = assign_crs(ds[name], crs='EPSG:4236')
ds = ds.rename(name)
ds.attrs['nodata'] = np.nan

ds = ds.odc.reproject(gbox, resampling='bilinear').compute()
ds = round_coords(ds)
ds = ds.rename(name)
ds = ds.where(mask)
ds['time'] = [pd.offsets.MonthBegin().rollback(t)+pd.Timedelta(14, 'd') for t in ds['time'].values]

In [ ]:
ds.astype('float32').to_netcdf('/g/data/os22/chad_tmp/Aus_CO2_fertilisation/data/SMrz_GLEAM_5km_1980_2023.nc')

In [ ]:
# import matplotlib.pyplot as plt
# ds.mean('time').plot(vmin=0.075, vmax=0.3)
# plt.title('GLEAM SMrz');

### Add SMrz to covars file

In [ ]:
import xarray as xr

In [ ]:
covar_file = f'/g/data/os22/chad_tmp/Aus_CO2_fertilisation/data/covars.nc'

In [ ]:
covariables =  xr.open_dataset(covar_file)
sm = xr.open_dataset('/g/data/os22/chad_tmp/Aus_CO2_fertilisation/data/SMrz_GLEAM_5km_1980_2023.nc')
sm = sm.sel(time=slice('1982','2022'))
covariables['smrz'] = sm['SMrz']

In [ ]:
covariables.to_netcdf('/g/data/os22/chad_tmp/Aus_CO2_fertilisation/data/covars__.nc')